In [7]:
import pandas as pd 
import numpy as np 
import snowflake.connector
import glob
import os


In [69]:
# Establish connection to Snowflake
conn = snowflake.connector.connect(
    user='GAANDAYA',
    password='Ashley@500@600!!!',
    account='boszpuz-awb44994'
)

# Create a cursor object to interact with the database
cs = conn.cursor()

In [70]:
#create warehouse and database using cs.execute commands
ware_house_name = 'MUSIC_WAREHOUSE'
database_name = 'SPOTIFY_SNOWFLAKE'
cs.execute(f"CREATE WAREHOUSE IF NOT EXISTS {ware_house_name}")
cs.execute(f"CREATE DATABASE IF NOT EXISTS {database_name}")

In [71]:
# Create CSV file format
cs.execute("use Database SPOTIFY_SNOWFLAKE")
query_for_csv_format = """ 
CREATE OR REPLACE FILE FORMAT csv_format
TYPE = 'CSV'
FIELD_OPTIONALLY_ENCLOSED_BY = '"'
"""
cs.execute(query_for_csv_format)


In [72]:
#Create the schema
schema_name = 'MUSIC_SCHEMA'

try:
    cs.execute(f"CREATE SCHEMA IF NOT EXISTS {schema_name}")
    print("Schema created successfully")
    
except Exception as e:
    print(f"An error occurred: {e}")



Schema created successfully


In [73]:
#Create a stage for staging edits etc.
stage_name ='spotify_stage'
try:
    cs.execute(f"CREATE OR REPLACE STAGE {schema_name}.{stage_name}")
    print("Stage created successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

Stage created successfully.


In [74]:
file_path = 'spotify_songs.csv'
spotify_df = pd.read_csv(file_path)
spotify_df.head()

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052


In [75]:
# conveert column to appropiate data type
spotify_df['track_album_release_date'] = pd.to_datetime(spotify_df['track_album_release_date'], errors='coerce')
print(spotify_df['track_album_release_date'].dtypes)

datetime64[ns]


In [76]:
spotify_df.dtypes

track_id                            object
track_name                          object
track_artist                        object
track_popularity                     int64
track_album_id                      object
track_album_name                    object
track_album_release_date    datetime64[ns]
playlist_name                       object
playlist_id                         object
playlist_genre                      object
playlist_subgenre                   object
danceability                       float64
energy                             float64
key                                  int64
loudness                           float64
mode                                 int64
speechiness                        float64
acousticness                       float64
instrumentalness                   float64
liveness                           float64
valence                            float64
tempo                              float64
duration_ms                          int64
dtype: obje

In [77]:
#Save new datatypes to the csv file
spotify_df.to_csv('spotify_songs.csv', index=False)


In [78]:
cs.execute(
    """
    CREATE OR REPLACE TABLE MUSIC_SCHEMA.spotify_songs (
        track_id VARCHAR,
        track_name VARCHAR,
        track_artist VARCHAR,
        track_popularity INT,
        track_album_id VARCHAR,
        track_album_name VARCHAR,
        track_album_release_date DATE,
        playlist_name VARCHAR,
        playlist_id VARCHAR,
        playlist_genre VARCHAR,
        playlist_subgenre VARCHAR,
        danceability FLOAT,
        energy FLOAT,
        key INT,
        loudness FLOAT,
        mode INT,
        speechiness FLOAT,
        acousticness FLOAT,
        instrumentalness FLOAT,
        liveness FLOAT,
        valence FLOAT,
        tempo FLOAT,
        duration_ms INT
    );
    """
)

In [79]:
#stage and load csv
csv_files = glob.glob(os.path.join("*.csv"))

cs.execute('use schema MUSIC_SCHEMA')

for file_path in csv_files:
    # The PUT command uploads a file to a stage
    cs.execute(f"PUT file://{file_path} @{schema_name}.{stage_name}")

# Confirm the files were uploaded
cs.execute(f"LIST @{schema_name}.{stage_name}")
for row in cs.fetchall():
    print(row)

('spotify_stage/spotify_songs.csv.gz', 3117728, 'edf6606786ec91883a7652e46c4ebb32', 'Fri, 1 Nov 2024 02:19:15 GMT')


In [80]:
# Load data from the staged file into the table
cs.execute("""
    COPY INTO MUSIC_SCHEMA.spotify_songs
    FROM @spotify_stage
    FILE_FORMAT = (TYPE = 'CSV', FIELD_OPTIONALLY_ENCLOSED_BY='"', SKIP_HEADER=1);
""")

print("Data loaded into the table successfully.")


Data loaded into the table successfully.


In [82]:
# Verify the load by selecting a few rows
cs.execute("SELECT * FROM MUSIC_SCHEMA.spotify_songs LIMIT 5")
for row in cs:
    print(row)


('6f807x0ima9a1j3VPbc7VN', "I Don't Care (with Justin Bieber) - Loud Luxury Remix", 'Ed Sheeran', 66, '2oCs0DGTsRO98Gh5ZSl2Cx', "I Don't Care (with Justin Bieber) [Loud Luxury Remix]", datetime.date(2019, 6, 14), 'Pop Remix', '37i9dQZF1DXcZDD7cfEKhW', 'pop', 'dance pop', 0.748, 0.916, 6, -2.634, 1, 0.0583, 0.102, 0.0, 0.0653, 0.518, 122.036, 194754)
('0r7CVbZTWZgbTCYdfa2P31', 'Memories - Dillon Francis Remix', 'Maroon 5', 67, '63rPSO264uRjW1X5E6cWv6', 'Memories (Dillon Francis Remix)', datetime.date(2019, 12, 13), 'Pop Remix', '37i9dQZF1DXcZDD7cfEKhW', 'pop', 'dance pop', 0.726, 0.815, 11, -4.969, 1, 0.0373, 0.0724, 0.00421, 0.357, 0.693, 99.972, 162600)
('1z1Hg7Vb0AhHDiEmnDE79l', 'All the Time - Don Diablo Remix', 'Zara Larsson', 70, '1HoSmj2eLcsrR0vE9gThr4', 'All the Time (Don Diablo Remix)', datetime.date(2019, 7, 5), 'Pop Remix', '37i9dQZF1DXcZDD7cfEKhW', 'pop', 'dance pop', 0.675, 0.931, 1, -3.432, 0, 0.0742, 0.0794, 2.33e-05, 0.11, 0.613, 124.008, 176616)
('75FpbthrwQmzHlBJLuGdC7

In [84]:
cs.close()
conn.close()